<a href="https://colab.research.google.com/github/UserMarcos/03MIAR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos(V2)_Marcos_Castro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Marcos Castro Castro  <br>
Url: https://github.com/UserMarcos/03MIAR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos(V2)_Marcos_Castro.ipynb<br>
Google Colab: https://colab.research.google.com/drive/1C3wA7QbGRMWOLkduwu2dsh6GIPmmDhJm?usp=sharing <br>
Problema:
>1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Configuración de Tribunales

Descripción del problema:(copiar enunciado)

## Problema 1. Organizar sesiones de doblaje(I)
Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:
- Número de actores: 10
- Número de tomas : 30
- Actores/Tomas : https://bit.ly/36D8IuK
  - 1 indica que el actor participa en la toma
  - 0 en caso contrario

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

## Representación del problema
El problema se representa con una lista de conjuntos. Cada conjunto es una toma, y contiene los números de actores que lo forman.

In [6]:
# Creamos la tabla de tomas
Numero_actores = 10

Actores_por_Toma = [
    {1, 2, 3, 4, 5},  # Toma 1
    {3, 4, 5},        # Toma 2
    {2, 5, 7},        # Toma 3
    {1, 2, 7, 8},     # Toma 4
    {2, 4, 8},        # Toma 5
    {1, 2, 4, 5},     # Toma 6
    {1, 2, 4, 5},     # Toma 7
    {1, 2, 6},        # Toma 8
    {1, 2, 4},        # Toma 9
    {1, 2, 6, 9},     # Toma 10
    {1, 2, 3, 5, 8},  # Toma 11
    {1, 2, 3, 4, 6},  # Toma 12
    {1, 4, 5},        # Toam 13
    {1, 3, 6},        # Toma 14
    {1, 2, 7},        # Toma 15
    {4, 10},          # Toma 16
    {1, 3},           # Toma 17
    {3, 6},           # Toma 18
    {1, 3},           # Toma 19
    {1, 3, 4, 5},     # Toma 20
    {6, 8},           # Toma 21
    {1, 2, 3, 4},     # Toma 22
    {1, 3},           # Toma 23
    {3, 6},           # Toma 24
    {1, 2, 4, 10},    # Toma 25
    {1, 3, 5, 9},     # Toma 26
    {4, 5},           # Toma 27
    {1, 4},           # Toma 28
    {1, 5, 6},        # Toma 29
    {1, 4}            # Toma 30
    ]

Aquí el contaje de cuantos actores hay por toma, y en cuantas tomas trabaja cada actor

In [7]:
N_Actores_por_toma = [len(x) for x in Actores_por_Toma]
print("Número de actores por toma: ", N_Actores_por_toma)

N_Tomas_por_actor = []
for i in range(Numero_actores):
  Numero_tomas = 0
  for Toma in Actores_por_Toma:
    if i+1 in Toma:
      Numero_tomas = Numero_tomas + 1
  N_Tomas_por_actor.append(Numero_tomas)

print("Número de tomas por actor: ", N_Tomas_por_actor)

Número de actores por toma:  [5, 3, 3, 4, 3, 4, 4, 3, 3, 4, 5, 5, 3, 3, 3, 2, 2, 2, 2, 4, 2, 4, 2, 2, 4, 4, 2, 2, 3, 2]
Número de tomas por actor:  [22, 14, 13, 15, 11, 8, 3, 4, 2, 2]


## Representación de la solucción
La solucción se representa como una lista de números enteros. Cada elemento representa una toma y el número es el día que se hará la toma. Así si tenemos
```S = [1, 1, 2, 2]```
, entendemos que las tomas 1 y 2 fueron el día 1 y las tomas 3 y 4 el día 2.

### Restricciones
La solucción no puede tener de 6 veces el mísmo número en la lista.

Hay que evitar que haya más de una forma de representar la misma solución. Si tenemos:
```
S1 = [1, 1, 2, 2]
S2 = [2, 2, 1, 1]
S3 = [4, 4, 0, 0]
```
Representan la misma solucción. Esto se puede evitar restringiendo el número de día que puede tener cada toma. Si tenemos una serie de N tomas, la toma nº _i_ (0<=i<N) puede tener el número de día:

* _i_
* Un número de día tal que en la subserie de la solucción de 0 a i-1 ya exista ese número de día más de 0 veces y menos de 6 veces.

Con estas restrinciones la única representación válida de las soluciones anteriores sería:


```
S = [0, 0, 2, 2]
```

En la siguiente función le damos como parámetro una lista de días de longitud _k_ tomas y nos devuelve una lista de días válidos que puede tomar la siguiente toma de la lista.

In [16]:
def Get_Posibles_Dias(Dias_Toma, indice):
  Hijos = []
  long = len(Dias_Toma)
  for i in range(indice):
    n = Dias_Toma.count(i)
    if n < 6 and n > 0:
      Hijos.append(i)
  Hijos.append(indice)
  return Hijos

Dia_por_Toma = [0, 0, 0, 0, 0, 0]
print(Get_Posibles_Dias(Dia_por_Toma, 7))

[7]


In [4]:
import random         #Para generar valores aleatorios
import math           #Modulo de funciones matematicas. Se usa para exp

La siguiente función crea una solucción aleatoria

In [17]:
def Crear_Solucion(N):
  solucion = []
  for i in range(N):
    solucion.append(random.choice(Get_Posibles_Dias(solucion, i)))
  return solucion

#Solucion = Crear_Solucion(30)
#print(Solucion)

Solucion = Crear_Solucion(30)
print(Solucion)

[0, 0, 2, 3, 2, 3, 6, 3, 3, 9, 10, 3, 0, 10, 0, 0, 16, 2, 16, 0, 9, 6, 22, 10, 16, 16, 3, 2, 16, 6]


Función para calcular el coste de una solucción de entrada

In [18]:
def Get_Actores_Dia(Dias_Toma, Actores_Toma):
  Actores_Dia = dict()
  for i in range(len(Dias_Toma)):
    N_Dia = Dias_Toma[i]
    if  N_Dia in Actores_Dia:
      Actores_Dia[N_Dia].update(Actores_Toma[i])
    else:
      Actores_Dia[N_Dia] = Actores_Toma[i].copy()
  return Actores_Dia

def Get_Coste(Dias_Toma, Actores_Toma):
  Actores_Dia = Get_Actores_Dia(Dias_Toma, Actores_Toma)
  Coste = 0
  for Dia in Actores_Dia:
    Coste += len(Actores_Dia[Dia])
  return Coste

Dia_por_Toma = [0, 1, 0, 3, 3]
Actores_Dia = Get_Actores_Dia(Dia_por_Toma, Actores_por_Toma)
print(Actores_Dia)
print(Get_Coste(Dia_por_Toma, Actores_por_Toma))

{0: {1, 2, 3, 4, 5, 7}, 1: {3, 4, 5}, 3: {1, 2, 4, 7, 8}}
14


El objetivo es encontrar la solucción que de el coste mínimo
#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

Si usáramos un algorítmo de fuerza bruta, probaríamos todas las combinaciones de sesiones repartidos en 5 días. Así para el primer día se puede hacer combinaciones de 6 sesiones de los 30 disponibles, en el segundo la combinación sería entre las 24 sesiones restantes, el tercer día entre 18, el cuarto entre 12 y el quinto día se queda con las 6 sesiones que quedan. Como el orden de los días no afecta al coste, en las combinaciones posibles se divide entre el número de permutaciones posibles de los 5 días.

\begin{align}
        \frac{C^6_{30} \cdot C^6_{24} \cdot C^6_{18} \cdot C^6_{12}}{P_5}=
        \frac{
          \begin{pmatrix}30 \\ 6\end{pmatrix} \cdot
          \begin{pmatrix}24 \\ 6\end{pmatrix} \cdot
          \begin{pmatrix}18 \\ 6\end{pmatrix} \cdot
          \begin{pmatrix}12 \\ 6\end{pmatrix}}
        {P_5} =
        \frac{
        \frac{30!}{24! \cdot 6!} \cdot
        \frac{24!}{18! \cdot 6!} \cdot
        \frac{18!}{12! \cdot 6!} \cdot
        \frac{12!}{6! \cdot 6!} }
        {5!}=
        \frac{30!}{(6!)^5 \cdot 5!}
    \end{align}


La búsqueda de la solucción optima tiene una complejidad $O(N) = N!$.

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Debido a que la complejidad es factorial habrá que usar una búsqueda hurística.

### Búsqueda local
Para la búsqueda local necesitamos crear vecinos. La siguiente función modifica la solucción cambiando el número de día en una de las tomas. No se comprueba si la solucción modificada tiene algún día con más de 6 tomas. Pero si puede modificar los número de días del resto de las tomas para que se cumpla el resto de las restricciones. La función devuelve el número de día en el que se convirtieron las tomas que estaban en el mismo día.

In [20]:
def Cambio_Dia(Dias_Toma, nToma, nDia):
  # Cambia la toma 'nToma' al día 'nDia'
  nP = -1
  dia_anterior = Dias_Toma[nToma]
  if (nDia <= nToma):
    Dias_Toma[nToma] = nDia
    for i in range(nToma+1, len(Dias_Toma)):
      if Dias_Toma[i] == nToma:
        if nP < 0:
          nP = i
        Dias_Toma[i] = nP
  else:
    Dias_Toma[nToma] = nToma
    for i in range(nToma+1, len(Dias_Toma)):
      if Dias_Toma[i] == nToma:
        if nP < 0:
          nP = i
        Dias_Toma[i] = nP

      if Dias_Toma[i] == nDia:
        Dias_Toma[i] = nToma
  if nP<0:
    nP = dia_anterior
  return nP

Dia_por_Toma = [0, 0, 0, 3, 3, 0, 6, 6, 3, 0, 10, 11, 6, 10, 6, 3, 3, 6, 6, 0, 10, 10, 3, 11, 11, 11, 10, 11, 11, 10]

#print(" ", Dia_por_Toma[:3], Get_Posibles_Dias(Dia_por_Toma[:3]))
print(" ", Dia_por_Toma)
n = Cambio_Dia(Dia_por_Toma, 3, 0)
print(n, Dia_por_Toma)
n = Cambio_Dia(Dia_por_Toma, 1, n)
print(n, Dia_por_Toma)

  [0, 0, 0, 3, 3, 0, 6, 6, 3, 0, 10, 11, 6, 10, 6, 3, 3, 6, 6, 0, 10, 10, 3, 11, 11, 11, 10, 11, 11, 10]
4 [0, 0, 0, 0, 4, 0, 6, 6, 4, 0, 10, 11, 6, 10, 6, 4, 4, 6, 6, 0, 10, 10, 4, 11, 11, 11, 10, 11, 11, 10]
0 [0, 1, 0, 0, 1, 0, 6, 6, 1, 0, 10, 11, 6, 10, 6, 1, 1, 6, 6, 0, 10, 10, 1, 11, 11, 11, 10, 11, 11, 10]


La siguiente función genera el mejor vecino

In [27]:
def genera_vecina(Dias_Toma, Actores_Toma):
  mejor_solucion = Dias_Toma
  mejor_coste = Get_Coste(Dias_Toma, Actores_Toma)
  N = len(Dias_Toma)
  for i in range(N):
    posibles = Get_Posibles_Dias(Dias_Toma, i)

    # Cambio de día la toma
    for dia in posibles:
      vecina = Dias_Toma.copy()
      Cambio_Dia(vecina, i, dia)
      vecina_coste = Get_Coste(vecina, Actores_Toma)

      if vecina_coste <= mejor_coste:
        mejor_coste = vecina_coste
        mejor_solucion = vecina

    #Intercambio de día con otra toma
    for j in range(i+1, N):
      # Intercambiamos dias
      vecina = Dias_Toma.copy()
      d = Cambio_Dia(vecina, i, vecina[j])
      Cambio_Dia(vecina, j, d)
      vecina_coste = Get_Coste(vecina, Actores_Toma)

      if vecina_coste <= mejor_coste:
        mejor_coste = vecina_coste
        mejor_solucion = vecina
  return mejor_solucion, mejor_coste

Dia_por_Toma = [0, 0, 0, 3, 3, 0, 6, 6, 3, 0, 10, 11, 6, 10, 6, 3, 3, 6, 6, 0, 10, 10, 3, 11, 11, 11, 10, 11, 11, 10]
#Crear_Solucion(len(Actores_por_Toma))
print("Solución inicial:", Dia_por_Toma)
print("Coste solución inicial: ", Get_Coste(Dia_por_Toma, Actores_por_Toma))

nueva_solucion, coste = genera_vecina(Dia_por_Toma, Actores_por_Toma)
print("Solución vecina: ", nueva_solucion)
print("Coste solución vecina: ", coste)

Solución inicial: [0, 0, 0, 3, 3, 0, 6, 6, 3, 0, 10, 11, 6, 10, 6, 3, 3, 6, 6, 0, 10, 10, 3, 11, 11, 11, 10, 11, 11, 10]
Coste solución inicial:  37
Solución vecina:  [0, 0, 0, 3, 3, 0, 6, 6, 3, 9, 10, 9, 6, 10, 6, 3, 3, 6, 6, 0, 10, 10, 3, 9, 9, 9, 10, 0, 9, 10]
Coste solución vecina:  35


In [28]:
# Búsqueda local
def Busqueda_Local(Actores_Toma):
  mejor_solucion = []

  # Generar una solucion inicial de referencia(aleatoria)
  Solucion_Referencia = Crear_Solucion(len(Actores_Toma))

  Mejor_Distancia = Get_Coste(Solucion_Referencia, Actores_Toma)

  #print("Solución inicial  es:" , Solucion_Referencia)
  #print("Distancia     :" , Mejor_Distancia)

  iteracion = 0            #Un contador para saber las iteraciones que hacemos
  while(iteracion < 2000):
    iteracion +=1         #Incrementamos el contador

     #Obtenemos la mejor vecina
    vecina, distancia_vecina = genera_vecina(Solucion_Referencia, Actores_Toma)

    #print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , vecina)
    #print("Distancia     :" , distancia_vecina)

    if distancia_vecina >= Mejor_Distancia:
      return mejor_solucion, Mejor_Distancia

    mejor_solucion = vecina
    Mejor_Distancia = distancia_vecina

    Solucion_Referencia = vecina

solucion, distancia = Busqueda_Local(Actores_por_Toma)
print(solucion)
print(distancia)

[0, 1, 2, 2, 0, 5, 0, 2, 8, 5, 0, 5, 5, 13, 2, 0, 13, 13, 18, 5, 2, 8, 18, 13, 0, 5, 1, 27, 2, 27]
34


Para evitar los mínimos locales se puede hacer varias llamadas a la búsqueda local y quedarse con el mínimo.

In [30]:
def busquedas_locales(Actores_Toma, Numero_busquedas):
  Mejor_Solucion = []
  Mejor_Distancia = 9999999

  for i in range(Numero_busquedas):
    solucion, distancia = Busqueda_Local(Actores_Toma)

    if Mejor_Distancia > distancia :
      Mejor_Distancia = distancia
      Mejor_Solucion = solucion
  return Mejor_Solucion, Mejor_Distancia

solucion, distancia = busquedas_locales(Actores_por_Toma, 100)
print(solucion)
print(distancia)

[0, 1, 2, 1, 1, 2, 2, 0, 8, 0, 1, 0, 2, 13, 2, 8, 13, 13, 13, 1, 20, 1, 13, 13, 8, 0, 2, 8, 0, 8]
28


# Recocido simulado

In [32]:
def genera_vecina_aleatorio(Dias_Toma):
  N = len(Dias_Toma)
  i = random.randint(0, N-1)

  posibles = Get_Posibles_Dias(Dias_Toma, i)
  #print(i, posibles)
  j = random.randint(0, len(posibles)-1)

  vecina = Dias_Toma.copy()
  Cambio_Dia(vecina, i, posibles[j])
  return vecina

Dia_por_Toma = Crear_Solucion(len(Actores_por_Toma))
print(Dia_por_Toma)
print("Coste solución inicial: ", Get_Coste(Dia_por_Toma, Actores_por_Toma))

nueva_solucion = genera_vecina_aleatorio(Dia_por_Toma)
print(nueva_solucion)
print("Coste solución vecina: ", Get_Coste(nueva_solucion, Actores_por_Toma))

[0, 1, 1, 3, 3, 0, 3, 3, 3, 0, 3, 11, 12, 13, 11, 15, 13, 11, 1, 0, 13, 1, 13, 11, 13, 15, 13, 27, 1, 12]
Coste solución inicial:  47
[0, 1, 1, 3, 3, 0, 3, 3, 3, 0, 3, 11, 12, 13, 11, 15, 13, 11, 1, 0, 13, 1, 13, 11, 13, 25, 13, 27, 1, 12]
Coste solución vecina:  47


In [33]:
#Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T)  :
    return True
  else:
    return False

#Funcion de descenso de temperatura
def bajar_temperatura(T):
  return T*0.99

In [34]:
def recocido_simulado(Actores_Toma, TEMPERATURA):
  mejor_solucion = []

  # Generar una solucion inicial de referencia(aleatoria)
  Solucion_Referencia = Crear_Solucion(len(Actores_Toma))
  #Solucion_Referencia = [0, 0, 2, 3, 4, 0, 2, 2, 0, 9, 10, 9, 9, 2, 10, 15, 0, 2, 15, 10, 4, 0, 2, 3, 24, 4, 3, 24, 28, 10]
  distancia_referencia = Get_Coste(Solucion_Referencia, Actores_Toma)

  mejor_solucion = []             #x* del seudocodigo
  mejor_distancia = 10e100        #F* del seudocodigo

  print("Solución inicial  es:" , Solucion_Referencia)
  print("Distancia     :" , distancia_referencia)

  iteracion = 0            #Un contador para saber las iteraciones que hacemos
  while TEMPERATURA > .0000001:
    iteracion +=1         #Incrementamos el contador

     #Obtenemos la mejor vecina ...
    vecina = genera_vecina_aleatorio(Solucion_Referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = Get_Coste(vecina, Actores_Toma)

    #Si es la mejor solución de todas se guarda(siempre!!!)
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      print("En la iteracion ", iteracion, ", la solución encontrada es:" , vecina)
      print("Distancia     :" , distancia_vecina)

    #Si la nueva vecina es mejor se cambia
    #Si es peor se cambia según una probabilidad que depende de T y delta(distancia_referencia - distancia_vecina)
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
      #solucion_referencia = copy.deepcopy(vecina)
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

     #Bajamos la temperatura
    TEMPERATURA = bajar_temperatura(TEMPERATURA)
  return mejor_solucion

sol = recocido_simulado(Actores_por_Toma, 10000000)
print(sol)

Solución inicial  es: [0, 1, 1, 1, 4, 5, 1, 5, 8, 0, 4, 4, 4, 0, 1, 8, 16, 1, 8, 4, 5, 8, 5, 5, 4, 8, 5, 8, 0, 16]
Distancia     : 40
En la iteracion  1 , la solución encontrada es: [0, 1, 1, 1, 4, 5, 1, 5, 8, 0, 4, 4, 4, 0, 1, 8, 16, 1, 8, 0, 5, 8, 5, 5, 4, 8, 5, 8, 0, 16]
Distancia     : 40
En la iteracion  26 , la solución encontrada es: [0, 1, 1, 1, 4, 5, 1, 5, 8, 0, 4, 4, 4, 0, 1, 8, 16, 1, 8, 4, 5, 8, 5, 5, 4, 0, 5, 8, 0, 16]
Distancia     : 38
[0, 1, 1, 1, 4, 5, 1, 5, 8, 0, 4, 4, 4, 0, 1, 8, 16, 1, 8, 4, 5, 8, 5, 5, 4, 0, 5, 8, 0, 16]


# Algorítmo genético

In [35]:
def generar_poblacion(Actores_Toma, N_Soluciones):
  poblacion_tmp = []
  N_Tomas = len(Actores_Toma)
  for i in range(N_Soluciones):
    solucion = Crear_Solucion(N_Tomas)
    distancia = Get_Coste(solucion, Actores_Toma)
    poblacion_tmp.append({'S':solucion, 'D':distancia})
  return poblacion_tmp

poblacion = generar_poblacion(Actores_por_Toma, 5)
print(poblacion)
poblacion.sort(key=lambda x:x['D'])
print(poblacion)

[{'S': [0, 0, 2, 3, 2, 5, 6, 2, 8, 9, 8, 6, 9, 13, 8, 8, 6, 0, 9, 0, 2, 3, 6, 6, 2, 2, 5, 6, 0, 13], 'D': 51}, {'S': [0, 0, 2, 0, 4, 4, 0, 7, 0, 7, 10, 4, 4, 13, 7, 15, 16, 17, 7, 4, 7, 2, 0, 4, 16, 10, 17, 17, 10, 29], 'D': 51}, {'S': [0, 0, 2, 3, 4, 2, 0, 3, 8, 3, 2, 11, 0, 3, 4, 3, 0, 17, 4, 11, 8, 8, 2, 3, 2, 4, 26, 17, 28, 17], 'D': 51}, {'S': [0, 1, 0, 0, 0, 0, 6, 7, 6, 9, 10, 1, 1, 0, 7, 1, 1, 7, 9, 6, 6, 7, 1, 10, 7, 6, 10, 9, 9, 9], 'D': 44}, {'S': [0, 1, 1, 3, 0, 0, 6, 7, 1, 6, 3, 7, 6, 1, 7, 6, 16, 0, 0, 1, 3, 0, 7, 6, 24, 1, 24, 27, 28, 16], 'D': 49}]
[{'S': [0, 1, 0, 0, 0, 0, 6, 7, 6, 9, 10, 1, 1, 0, 7, 1, 1, 7, 9, 6, 6, 7, 1, 10, 7, 6, 10, 9, 9, 9], 'D': 44}, {'S': [0, 1, 1, 3, 0, 0, 6, 7, 1, 6, 3, 7, 6, 1, 7, 6, 16, 0, 0, 1, 3, 0, 7, 6, 24, 1, 24, 27, 28, 16], 'D': 49}, {'S': [0, 0, 2, 3, 2, 5, 6, 2, 8, 9, 8, 6, 9, 13, 8, 8, 6, 0, 9, 0, 2, 3, 6, 6, 2, 2, 5, 6, 0, 13], 'D': 51}, {'S': [0, 0, 2, 0, 4, 4, 0, 7, 0, 7, 10, 4, 4, 13, 7, 15, 16, 17, 7, 4, 7, 2, 0, 4, 16, 10, 17

In [36]:
#Evalua la población y devuelve el mejor individuo
def Evaluar_Poblacion(poblacion):
  mejor_solucion = []
  mejor_distancia = 999e999
  for solucion in poblacion:
    distancia = solucion['D'] #Get_Coste(Get_Actores_Dia(solucion, Actores_Toma))
    if distancia < mejor_distancia:
      mejor_distancia = distancia
      mejor_solucion = solucion['S']
  return mejor_solucion, mejor_distancia

print(Evaluar_Poblacion(poblacion))

([0, 1, 0, 0, 0, 0, 6, 7, 6, 9, 10, 1, 1, 0, 7, 1, 1, 7, 9, 6, 6, 7, 1, 10, 7, 6, 10, 9, 9, 9], 44)


In [37]:
#Funcion para generar hijos a partir de 2 padres:
# Se elige el metodo de 1-punto de corte pero es posible usar otros n-puntos, uniforme, dependiendo del problema
def Descendencia(padre1, padre2, mutacion):
  longitud = len(padre1)
  punto_corte = 5 #random.randint(0, longitud - 1)

  hijo1 = [-1] * longitud
  hijo1[:punto_corte] = padre1[:punto_corte]
  for i in range(punto_corte, longitud):
    if padre2[i] < punto_corte:
      hijo1[i] = padre1[padre2[i]]
    else:
      hijo1[i] = padre2[i]

  # Vemos si hacemos mutación
  if (mutacion < random.random()):
    genera_vecina_aleatorio(hijo1)

  return hijo1

sol1 = Crear_Solucion(30)
sol2 = Crear_Solucion(30)

hijo = Descendencia(sol1, sol2, 0.1)
print("Padre 1: ", sol1)
print("Padre 2: ", sol2)
print("Hijo   : ", hijo)

Padre 1:  [0, 0, 0, 3, 4, 3, 4, 3, 4, 9, 3, 11, 3, 11, 4, 11, 9, 11, 9, 9, 0, 21, 9, 4, 9, 21, 4, 3, 11, 0]
Padre 2:  [0, 0, 0, 0, 4, 5, 4, 4, 4, 0, 10, 10, 5, 5, 0, 15, 5, 4, 15, 4, 15, 5, 15, 5, 15, 15, 26, 10, 28, 28]
Hijo   :  [0, 0, 0, 3, 4, 5, 4, 4, 4, 0, 10, 10, 5, 5, 0, 15, 5, 4, 15, 4, 15, 5, 15, 5, 15, 15, 26, 10, 28, 28]


In [38]:
#Para el operador de cruce 1-punto los hijos generados no son soluciones(algunos nodos se repiten y otros no están)
def Factibilizar(solucion):
  for i in range(len(solucion)-6, -1, -1):
    dia = solucion[i]
    N = solucion[i:].count(dia)
    if N == 6:
      solucion[i:] = [x if x != dia else i for x in solucion[i:]]

  # dias = tuple(set(solucion))
  # for d in dias:
  #   N = solucion.count(d)
  #   if N > 6:
  #     return False
  # return True

sol = [0] * 10
print(sol)
Factibilizar(sol)
print(sol)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 4, 4, 4, 4, 4, 4]


In [39]:
#Funcion de cruce. Recibe una poblacion(lista de soluciones) y devuelve la población ampliada con los hijos.
# Todos los individuos de la población son selecionados para el cruce(si la población es par)
# Podría aplicarse un proceso previo de selección para elegir los individuos que se desea cruzar.
def Cruzar(poblacion, mutacion, Actores_Toma):
  N = len(poblacion)
  lista = random.sample(range(N), N)
  for i in range(N//2):
    sol1 = poblacion[lista[i*2]]['S']
    sol2 = poblacion[lista[i*2+1]]['S']
    hijo = Descendencia(sol1, sol2, mutacion)
    #print(hijo)
    Factibilizar(hijo)

    # if Factibilizar(hijo):
    distancia = Get_Coste(hijo, Actores_Toma)
    poblacion.append({'S':hijo, 'D':distancia})

    # hijo = Descendencia(sol2, sol1, mutacion)
    # #print(hijo)
    # Factibilizar(hijo)

    # # if Factibilizar(hijo):
    # distancia = Get_Coste(Get_Actores_Dia(hijo, Actores_Toma))
    # poblacion.append({'S':hijo, 'D':distancia})

poblacion = generar_poblacion(Actores_por_Toma, 500)
print(len(poblacion))
Cruzar(poblacion, 0.0, Actores_por_Toma)
print(len(poblacion))

500
750


In [40]:
#Funcion de seleccion de la población. Recibe como parametro una poblacion y
# devuelve una poblacion a la que se ha eliminado individuos poco aptos(fitness alto) y para mantener una poblacion estable de N individuos
#Se tiene en cuenta el porcentaje elitismo pasado como parametro
# Para los individuos que no son de la elite podríamos usar una selección de ruleta(proporcional a su fitness)
def Seleccionar(poblacion, N, elitismo):
  # Ordenamos la población
  poblacion.sort(key=lambda x:x['D'])

  N_Elite = int(N*elitismo)

  while(len(poblacion)>N):
    i = max (random.randrange(len(poblacion)), random.randrange(len(poblacion)))
    if i > N_Elite:
      del poblacion[i]


poblacion = generar_poblacion(Actores_por_Toma, 8)
print(len(poblacion))
Seleccionar(poblacion, 5, 0.2)
print(poblacion)

8
[{'S': [0, 1, 0, 0, 0, 1, 6, 7, 7, 6, 6, 0, 1, 7, 1, 15, 7, 7, 18, 6, 7, 18, 0, 18, 18, 6, 15, 15, 15, 1], 'D': 39}, {'S': [0, 1, 0, 3, 0, 5, 1, 1, 3, 5, 0, 5, 12, 0, 12, 0, 16, 1, 16, 16, 3, 12, 12, 1, 12, 5, 5, 12, 1, 29], 'D': 41}, {'S': [0, 1, 2, 0, 0, 0, 6, 1, 2, 1, 0, 1, 1, 13, 0, 1, 13, 6, 18, 18, 18, 13, 18, 23, 18, 13, 13, 2, 18, 29], 'D': 45}, {'S': [0, 0, 0, 3, 3, 3, 3, 0, 0, 3, 10, 0, 10, 3, 10, 15, 10, 10, 15, 10, 20, 20, 22, 23, 15, 20, 22, 15, 22, 23], 'D': 46}, {'S': [0, 0, 0, 0, 4, 4, 6, 6, 8, 9, 6, 8, 6, 6, 9, 9, 4, 8, 18, 0, 18, 21, 0, 6, 4, 25, 18, 25, 9, 4], 'D': 49}]


Proceso principal

In [41]:
#Funcion principal del algoritmo genetico
#######################################################3
def algoritmo_genetico(Actores_Toma, N=100, mutacion=.15, elitismo=.1, generaciones=100):
  # problem = datos del problema
  # N = Tamaño de la población
  # mutacion = probabilidad de una mutación
  # elitismo = porcion de la mejor poblacion a mantener
  # generaciones = nº de generaciones a generar para finalizar

  #Genera la poblacion inicial
  #Nodos = list(problem.get_nodes())
  poblacion = []
  poblacion = generar_poblacion(Actores_Toma, N)

  #Inicializamos valores para la mejor solucion
  (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion)


  #Condicion de parada
  parar = False
  n=0
  #Inciamos el cliclo de generaciones
  while(parar == False) :

    #Cruce de la poblacion(incluye mutación)
    #poblacion =
    Cruzar(poblacion, mutacion, Actores_Toma)

    #print("Tamaño de población : ", len(poblacion))

    #Seleccionamos la población
    #poblacion =
    Seleccionar(poblacion, N, elitismo)

    #Evaluamos la nueva población
    (mejor_solucion, mejor_distancia) = Evaluar_Poblacion(poblacion)

    print("Generacion #", n, " - La mejor solución es:" , mejor_solucion, " con distancia " , mejor_distancia)

    #Numero de generaciones. Criterio de parada
    if n==generaciones:
      parar = True
    n +=1

  return mejor_solucion


sol = algoritmo_genetico(Actores_Toma=Actores_por_Toma, N=200, mutacion=.5, elitismo=.10, generaciones=200)

Generacion # 0  - La mejor solución es: [0, 0, 0, 0, 0, 5, 5, 0, 8, 8, 5, 8, 5, 13, 13, 5, 5, 13, 8, 13, 20, 8, 13, 20, 8, 20, 26, 13, 20, 20]  con distancia  38
Generacion # 1  - La mejor solución es: [0, 0, 0, 0, 0, 5, 5, 0, 8, 8, 5, 8, 5, 13, 13, 5, 5, 13, 8, 13, 20, 8, 13, 20, 8, 20, 26, 13, 20, 20]  con distancia  38
Generacion # 2  - La mejor solución es: [0, 0, 0, 0, 0, 5, 5, 0, 8, 8, 5, 8, 5, 13, 13, 5, 5, 13, 8, 13, 20, 8, 13, 20, 8, 20, 26, 13, 20, 20]  con distancia  38
Generacion # 3  - La mejor solución es: [0, 0, 0, 0, 0, 5, 5, 0, 8, 8, 5, 8, 5, 13, 13, 5, 5, 13, 8, 13, 20, 8, 13, 20, 8, 20, 26, 13, 20, 20]  con distancia  38
Generacion # 4  - La mejor solución es: [0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 6, 6, 12, 12, 14, 15, 12, 14, 15, 15, 12, 12, 14, 12, 14, 14, 15, 15, 15, 14]  con distancia  37
Generacion # 5  - La mejor solución es: [0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 6, 6, 12, 12, 14, 15, 12, 14, 15, 15, 12, 12, 14, 12, 14, 14, 15, 15, 15, 14]  con distancia  37
Generacion # 6  